# Matrix Factorization

In this task you are supposed to (manually) implement the matrix factorization variant you learned in the Data Cleaning chapter using the `numpy` library.

In [3]:
import numpy as np

We continue the scenario from the tutorials.

Assume that you have a ginormous database $D$ of three users and three movies and ratings provided by some users to some movies, which we represent as a matrix, where the entry $D_{ij}$ represents the rating user $i$ gave to movie $j$.
Since not all users have rated movies, and the rating ranges from 1 to 5, we encode missing ratings as 0.

In [5]:
# missing values encoded as 0
# 3x3 Matrix
D = [
     [3,1,0],
     [1,0,3],
     [0,3,5],
    ]
D = np.array(D)
n_users = D.shape[0]
n_movies = D.shape[1]

First, randomly initialize the two factors $E$ and $A$ for $f=2$ latent features. For evaluating the correctness of your results from the tutorial, you may *additionally* provide hard-coded inital factors as they have been provided in the tutorial.

In [110]:
# number of latent features
lr = 0.001
epochs = 5000
f = 2
# Random matrix E is N*f, so 3x3
E = np.random.normal(0, .1, (n_users,f))
# E = np.array([[0.34, 0.67], [0.38, 0.93], [0.21, 0.39]])

#Random matrix A is f*M, so 3x3
A = np.random.normal(0, .1, (f,n_movies))
# A = np.array([[0.45, 0.78, 0.37], [0.02, 0.67, 0.35]])
masks_zeros = (D == 0)

for i in range(epochs):
    d = E @ A
    # print("D\n", D)
    # print("d\n", d)
    # print("E\n", E)
    # print("D\n", D)

    diff = 2 * (D - d)
    diff[masks_zeros] = D[masks_zeros]
    # print("diff masked\n", diff)

    e_errors = diff @ A.T
    a_errors = E.T @ diff

    E = E + (e_errors * lr)
    A = A + (a_errors * lr)
    # print("e_err\n", e_errors)
    # print("a_err\n", a_errors)

d,D, diff

(array([[2.99999905, 1.00000273, 1.98532047],
        [1.00000055, 2.01132515, 2.99999924],
        [4.15443932, 2.99999838, 5.00000078]]),
 array([[3, 1, 0],
        [1, 0, 3],
        [0, 3, 5]]),
 array([[ 1.90378196e-06, -5.45933415e-06,  0.00000000e+00],
        [-1.09592717e-06,  0.00000000e+00,  1.52314402e-06],
        [ 0.00000000e+00,  3.23687119e-06, -1.55025938e-06]]))

Implement a function that takes the data matrix $D$, the inital factors $E, A$, the number of epochs (iterations), the learning rate $\eta$, and performs the factorization of $D$. Use a default number of 5000 for the epochs and 0.001 for $\eta$.

Updates to $E$ and $A$ are applied immediately. $\tilde{D}$ is updated after an entry from D was completely dealt with. Update ordered by latent features and E before A.

In [66]:
def cmpt(matrixD, facE, facA, iter = 5000, rate = 0.001):
    
    

SyntaxError: incomplete input (904152585.py, line 3)

Now test your matrix factorization for the parameters sepcified above.

In [67]:
# TODO your code goes here